In [1]:
import boto3
import json
import pandas as pd
import redshift_connector

In [2]:
def get_secret(session):
    client = session.client(
        service_name='secretsmanager',
        region_name="ap-southeast-1"
    )
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId="prod/mitrphol-redshift-prod/mitrphol-smartfactory"
        )
    except ClientError as e:
        raise e

    # Decrypts secret using the associated KMS key.
    secret = get_secret_value_response['SecretString']
    return json.loads(secret)

In [3]:
session = boto3.session.Session()
secret = get_secret(session)

NameError: name 'ClientError' is not defined

In [ ]:
conn = redshift_connector.connect(
    host     = secret['host'],
    database = secret['database'],
    user     = secret['username'],
    password = secret['password']
)
cursor: redshift_connector.Cursor = conn.cursor()
cursor.execute("select * from mitrphol_prod.smart_factory.shredder_dim_taginfo where factory='MAC' ")
df_taginfo: pd.DataFrame = cursor.fetch_dataframe()

cursor.execute("select * from mitrphol_prod.smart_factory.shredder_fact_pidata_mac")
df_data: pd.DataFrame = cursor.fetch_dataframe()